In [1]:
# To run on Kaggle platform
!pip install conllu
!pip install wandb

In [2]:
from transformers import AutoTokenizer, XLMRobertaModel
import datasets
import torch
import sys
from datasets import Dataset

In [3]:
# provided dependency relations
all_deprels = [
    # these are the default UD dependency relations according to https://universaldependencies.org/u/dep/
    "acl", "acl:relcl", "advcl", "advcl:relcl", "advmod", "advmod:emph", "advmod:lmod", "amod", "appos",
    "aux", "aux:pass", "case", "cc", "cc:preconj", "ccomp", "clf", "compound", "compound:lvc",
    "compound:prt", "compound:redup", "compound:svc", "conj", "cop", "csubj", "csubj:outer",
    "csubj:pass", "dep", "det", "det:numgov", "det:nummod", "det:poss", "discourse", "dislocated",
    "expl", "expl:impers", "expl:pass", "expl:pv", "fixed", "flat", "flat:foreign", "flat:name",
    "goeswith", "iobj", "list", "mark", "nmod", "nmod:poss", "nmod:tmod", "nsubj", "nsubj:outer",
    "nsubj:pass", "nummod", "nummod:gov", "obj", "obl", "obl:agent", "obl:arg", "obl:lmod",
    "obl:tmod", "orphan", "parataxis", "punct", "reparandum", "root", "vocative", "xcomp",

    # we need some more for en_ewt
    "det:predet", "obl:npmod", "nmod:npmod"
]

# map dependency relations to IDs
deprel_to_id = dict((rel, idx) for idx, rel in enumerate(all_deprels))

In [4]:
# provided code for the assignment from tokenize-for-parsing.py

# Code for the assignment in https://github.com/coli-saar/cl/wiki/Assignment:-Dependency-parsing
# Alexander Koller, December 2023

def strip_none_heads(examples, i):
    tokens = examples["tokens"][i]
    heads = examples["head"][i]
    deprels = examples["deprel"][i]

    non_none = [(t, h, d) for t, h, d in zip(tokens, heads, deprels) if h != "None"]
    return zip(*non_none)


def map_first_occurrence(nums):
    """
    Maps a list of numbers to a dictionary that assigns each unique number the position of its first occurrence.

    Example:
    > map_first_occurrence([0,1,2,3,3,3,4])
    {0: 0, 1: 1, 2: 2, 3: 3, 4: 6}

    :param nums:
    :return:
    """
    seen = set()
    return {num: i for i, num in enumerate(nums) if num is not None and num not in seen and not seen.add(num)}


def pad_to_same_size(lists, padding_symbol):
    maxlen = max([len(l) for l in lists])
    return [l + (padding_symbol,) * (maxlen - len(l)) for l in lists]


def tokenize_and_align_labels(examples, deprel_to_id, tokenizer, skip_index=-100):
    # delete tokens with "None" head and their annotations
    examples_tokens, examples_heads, examples_deprels = [], [], []
    for sentence_id in range(len(examples["tokens"])):
        tt, hh, dd = strip_none_heads(examples, sentence_id)
        examples_tokens.append(tt)
        examples_heads.append(hh)
        examples_deprels.append(dd)

    tokenized_inputs = tokenizer(examples_tokens, truncation=True, is_split_into_words=True,
                                 padding=True)  # get "tokenizer" from global variable
    # tokenized_inputs is a dictionary with keys input_ids and attention_mask;
    # each is a list (per sentence) of lists (per token).

    remapped_heads = []  # these will be lists (per sentence) of lists (per token)
    deprel_ids = []
    tokens_representing_words = []
    num_words: list[int] = []
    maxlen_t2w = 0  # max length of a token_to_word_here list

    for sentence_id, annotated_heads in enumerate(examples_heads):
        deprels = examples_deprels[sentence_id]
        word_ids = tokenized_inputs.word_ids(batch_index=sentence_id)
        word_pos_to_token_pos = map_first_occurrence(
            word_ids)  # word-pos to first token-pos; both start at 0 for first word (actual) / first token (BOS)

        previous_word_idx = None
        heads_here: list[int] = []
        deprel_ids_here: list[int] = []

        # list of token positions that map to words (first token of each word)
        # token 0 -> word 0 (BOS)
        tokens_representing_word_here: list[int] = [0]

        for sentence_position, word_idx in enumerate(word_ids):
            # Special tokens (BOS, EOS) have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                heads_here.append(skip_index)
                deprel_ids_here.append(skip_index)

            # We set the label for the first token of each word;
            # subsequent tokens of the same word will have the same word_idx.
            elif word_idx != previous_word_idx:
                if annotated_heads[word_idx] == "None":  # added by padding
                    print("A 'None' head survived!")
                    sys.exit(0)
                else:
                    # Map HEAD annotation to position of first token of head word.
                    # HEAD = 0 => map it to first token (BOS)
                    # Otherwise, look up first token for HEAD-1 (HEAD is 1-based, word positions are 0-based)
                    head_word_pos = int(annotated_heads[word_idx])
                    head_token_pos = 0 if head_word_pos == 0 else word_pos_to_token_pos[head_word_pos - 1]

                    heads_here.append(head_token_pos)
                    deprel_ids_here.append(deprel_to_id[deprels[word_idx]])

                    tokens_representing_word_here.append(sentence_position)  # first word is index 1; index 0 is BOS

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                heads_here.append(skip_index)
                deprel_ids_here.append(skip_index)

            previous_word_idx = word_idx

        remapped_heads.append(heads_here)
        deprel_ids.append(deprel_ids_here)
        tokens_representing_words.append(tokens_representing_word_here)

        num_words.append(len(tokens_representing_word_here))
        if len(tokens_representing_word_here) > maxlen_t2w:
            maxlen_t2w = len(tokens_representing_word_here)

    # pad t2w lists to same length
    for t2w in tokens_representing_words:
        t2w += [-1] * (maxlen_t2w - len(t2w))

    tokenized_inputs["head"] = remapped_heads
    tokenized_inputs["deprel_ids"] = deprel_ids
    tokenized_inputs["tokens_representing_words"] = tokens_representing_words
    tokenized_inputs["num_words"] = num_words
    tokenized_inputs["tokenid_to_wordid"] = [tokenized_inputs.word_ids(batch_index=i) for i in
                                             range(len(examples_heads))]  # map token ID to word ID

    return tokenized_inputs

In [5]:
# load the dataset
dataset = datasets.load_dataset(path="universal_dependencies", name="en_ewt", trust_remote_code=True)

# datasets for training, validation and testing
train_dataset = dataset["train"]
# test dataset
test_dataset = dataset["test"]
# validation dataset
val_dataset = dataset["validation"]


README.md:   0%|          | 0.00/191k [00:00<?, ?B/s]

universal_dependencies.py:   0%|          | 0.00/87.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2077 [00:00<?, ? examples/s]

In [6]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

# Test tokenization
def test_tokenization_and_alignment(dataset, deprel_to_id, tokenizer, num_examples=10):
    tokenized_inputs = tokenize_and_align_labels(dataset[:num_examples], deprel_to_id, tokenizer)

    for num in range(num_examples):
        tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][num])
        word_ids = tokenized_inputs["tokenid_to_wordid"][num]
        # example header
        print(f"-----------------------This is example {num + 1}--------------------------")
        
        # print each token and its corresponding values
        for idx, token in enumerate(tokens):
            # Break if padding token is reached
            if token == tokenizer.pad_token:
                break
            # Get word mapping
            map_word = word_ids[idx]
            
            # Get values for different columns
            input_ids = tokenized_inputs["input_ids"][num][idx]
            deprel_id = tokenized_inputs["deprel_ids"][num][idx]
            head = tokenized_inputs["head"][num][idx]
            # -100 -> None
            if deprel_id != -100:
                deprel = all_deprels[deprel_id]
            else:
                deprel = "None"
                
            # if token is None, set it to "None"
            if token:
                token_str = token
            else:
                token_str = "None"
            
            token_id_str = str(input_ids)
            head_stringed = str(head)
            deprel_str = deprel if deprel else "None"
            if map_word is not None:
                stringed_map_word = str(map_word)
            else:
                stringed_map_word = "None"

            print(f"{token_str}   {token_id_str}   {head_stringed}   {deprel_str}   {stringed_map_word}")

# Example output
test_tokenization_and_alignment(train_dataset, deprel_to_id, tokenizer)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

-----------------------This is example 1--------------------------
<s>   0   -100   None   None
▁Al   884   0   root   0
▁-   20   1   punct   1
▁Zaman   53113   1   flat   2
▁:   152   1   punct   3
▁American   15672   6   amod   4
▁forces   84616   7   nsubj   5
▁killed   152388   1   parataxis   6
▁Sha   7224   7   obj   7
ikh   41336   -100   None   7
▁Abdullah   34490   8   flat   8
▁al   144   8   flat   9
▁-   20   8   punct   10
▁Ani   32340   8   flat   11
▁   6   8   punct   12
,   4   -100   None   12
▁the   70   17   det   13
▁prea   19542   8   appos   14
cher   5372   -100   None   14
▁at   99   21   case   15
▁the   70   21   det   16
▁mos   7304   7   obl   17
que   944   -100   None   17
▁in   23   25   case   18
▁the   70   25   det   19
▁town   59444   21   nmod   20
▁of   111   27   case   21
▁Qa   16785   25   nmod   22
im   464   -100   None   22
▁   6   25   punct   23
,   4   -100   None   23
▁near   43573   35   case   24
▁the   70   35   det   25
▁Syria   5171

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# this cell to get the dataloader

batch_size = 32

# train dataset, [:]
train_inputs = tokenize_and_align_labels(train_dataset[:], deprel_to_id, tokenizer)
# Convert BatchEncoding to Dataset
train_dataset = Dataset.from_dict(train_inputs.data)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'head', 'deprel_ids'])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size)

# test dataset
test_inputs = tokenize_and_align_labels(test_dataset[:], deprel_to_id, tokenizer)
test_dataset = Dataset.from_dict(test_inputs.data)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'head', 'deprel_ids'])
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size)

# validation dataset
valid_inputs = tokenize_and_align_labels(val_dataset[:], deprel_to_id, tokenizer)
val_dataset = Dataset.from_dict(valid_inputs.data)
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'head', 'deprel_ids'])
valid_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size= batch_size)


In [8]:
class DependencyParserModel(torch.nn.Module):
    def __init__(self,
                 hidden_size=768,
                 edge_dim=500, 
                 edge_pred=True,
                 dropout=0.3
                 ):
        
        super().__init__()
        # parameters
        self.roberta_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.edge_pred = edge_pred

        # freeze all the parameters 
        for param in self.roberta_model.parameters():
            param.requires_grad = False


        #  define for edge head and dependency projection
        if self.edge_pred:
            self.edge_mlp_head = torch.nn.Sequential(
                torch.nn.Linear(hidden_size, edge_dim),
                torch.nn.ReLU(),
                torch.nn.Dropout(dropout))
            
            self.edge_mlp_dep = torch.nn.Sequential(
                torch.nn.Linear(hidden_size, edge_dim),
                torch.nn.ReLU(),
                torch.nn.Dropout(dropout))

            # u1 for edge scoring
            self.u1 = torch.nn.Parameter(torch.empty(edge_dim, edge_dim))
            torch.nn.init.xavier_uniform_(self.u1)
            # u2
            self.u2 = torch.nn.Parameter(torch.empty(edge_dim))
            torch.nn.init.xavier_uniform_(self.u2.unsqueeze(0))
            self.u2.squeeze(0)


    def forward(self, input_ids, attention_mask):
        
        h_head = None
        h_dep = None
        
        outputs = self.roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        if self.edge_pred:
            # project
            h_head = self.edge_mlp_head(last_hidden_state)
            h_dep = self.edge_mlp_dep(last_hidden_state)

        return h_head, h_dep
            

    def score_edges(self, h_head, h_dep):
        #  to compute score[i, j]
        u1_h_head = torch.einsum("bsd,dd->bsd", h_head, self.u1)
        u2_h_head = torch.einsum("bid,d->bi", h_head, self.u2)
        h_head_11_h_dep = torch.einsum("bim,bjm->bij", u1_h_head, h_dep)
        
        scores = h_head_11_h_dep + u2_h_head.unsqueeze(2)
        return scores




In [9]:
import wandb
# you wandb api key
wandb.login(key="YOUR API KEY")

# train the model
def train(model, train_dataloader, valid_dataloader, device, num_epochs=8, lr=3e-3, weight_decay=8e-3):
    
    # wandb starts
    wandb.init(project="project 5 for cl", name="assignment 5", resume=False, config={
        "learning_rate": lr,
        "weight_decay": weight_decay,
        "num_epochs": num_epochs
    })

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=weight_decay, lr=lr)
    # ignore -100
    criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
    val_dataset = next(iter(valid_dataloader))
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        # initialize loss
        total_loss = 0.0
        # for batch
        for idx, data in enumerate(train_dataloader):
            # to device
            input_id_data = data["input_ids"].to(device)
            head_data = data["head"].to(device)
            att_mask = data["attention_mask"].to(device)
            
            # to zero the parameter gradients
            optimizer.zero_grad()
            
            # to get the output
            h_h, h_d = model(input_id_data, att_mask)
            
            # to score the edges
            e_score = model.score_edges(h_h, h_d)
           
            # to get loss
            e_score_flat = e_score.view(-1, e_score.size(-1))
            head_data_flat = head_data.view(-1)
            loss = criterion(e_score_flat, head_data_flat)

            
            # backward
            loss.backward()
            optimizer.step()

            # to get Loss curve data
            total_loss += loss.item()
            print(f"In Epoch {epoch}, Present Iteration {idx}, Loss is: {loss.item()}")
            # to get the loss
            wandb.log({"TRAIN LOSS": loss.item()})
            
            # every 100 iteration,evaluate
            if idx % 100 == 0 and idx > 0:
                # evaluate now
                model.eval()
                with torch.no_grad():
                    input_id_data = val_dataset["input_ids"].to(device)
                    att_mask = val_dataset["attention_mask"].to(device)
                    head_data = val_dataset["head"].to(device)
                    # get score
                    h_h, h_d = model(input_id_data, att_mask)
                    e_score = model.score_edges(h_h, h_d)
                    # get the loss
                    valid_loss = criterion(e_score.view(-1, e_score.size(-1)), head_data.view(-1))
                    # show the loss
                    print(f"Epoch {epoch + 1}, Batch {idx}, Valid Loss: {valid_loss}")
                    wandb.log({"Valid_Loss is ": valid_loss.item()})
                model.train()
        # to see the trend of loss averaged
        loss_average = total_loss / len(train_dataloader)
        print(f"The averaged loss of Epoch {epoch + 1} / {num_epochs} is {loss_average}.")

        # prediction on validation set
        model.eval()
        # total and right head tagging
        total_count = 0 
        total_correct = 0
        with torch.no_grad():
            for idx, data in enumerate(valid_dataloader):
                input_id_data = data["input_ids"].to(device)
                att_mask = data["attention_mask"].to(device)
                head_data = data["head"].to(device)

                h_h, h_d = model(input_id_data, att_mask)
                # choose highest score here
                e_score = model.score_edges(h_h, h_d)
                best_heads = e_score.argmax(dim=-1) 

               # to get valid mask & count correct
                val_position = (head_data != -100) & (att_mask == 1)
                total_correct += ((best_heads == head_data) & val_position).sum().item()
                total_count += val_position.sum().item()
        # back to train mode
        model.train()
        wandb.log({"Accuracy_Head_Tagging": total_correct / total_count})
    # finish
    wandb.finish()
    # save
    torch.save(model.state_dict(), "dependency_parser.pth")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: freemensword (freemensword-saarland-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# below about 17 min on P100

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = DependencyParserModel()
train(model, train_dataloader, valid_dataloader, device, lr=6e-4, num_epochs=10)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In Epoch 0, Present Iteration 0, Loss is: 5.277841091156006
In Epoch 0, Present Iteration 1, Loss is: 5.215796947479248
In Epoch 0, Present Iteration 2, Loss is: 5.179658889770508
In Epoch 0, Present Iteration 3, Loss is: 5.1277031898498535
In Epoch 0, Present Iteration 4, Loss is: 5.027556896209717
In Epoch 0, Present Iteration 5, Loss is: 4.989219665527344
In Epoch 0, Present Iteration 6, Loss is: 4.936363220214844
In Epoch 0, Present Iteration 7, Loss is: 4.8315839767456055
In Epoch 0, Present Iteration 8, Loss is: 4.8482890129089355
In Epoch 0, Present Iteration 9, Loss is: 4.789571285247803
In Epoch 0, Present Iteration 10, Loss is: 4.680203437805176
In Epoch 0, Present Iteration 11, Loss is: 4.465713024139404
In Epoch 0, Present Iteration 12, Loss is: 4.531825542449951
In Epoch 0, Present Iteration 13, Loss is: 4.606533527374268
In Epoch 0, Present Iteration 14, Loss is: 4.408348083496094
In Epoch 0, Present Iteration 15, Loss is: 4.337803840637207
In Epoch 0, Present Iteration 1

Some observeations:
The loss decreases gradually, but the accuracy 0.74706 seems not good enough. Maybe more layers should be added and more complex layers should be added to get better predictions.